In [5]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd  

In [6]:
engine = create_engine(connstr) 
def get_df(query):
    conn = engine.connect() 
    df = pd.read_sql(query,con=conn) 
    conn.close()
    return df

In [92]:
def cspan(IDTEXT): 
    span = 6
    if IDTEXT[0] == '0':
        return '6', '' 
    IDTEXT=IDTEXT.split('.')
    if len(IDTEXT) == 1:
        return '3','' 
    span = 4-len(IDTEXT)
    if span < 0:
        span=0    
    gap = len(IDTEXT)-1
    if gap < 0:
        gap=0    
    ele = f'<td colspan="{gap}">&nbsp</td>'
    return str(span), ele
cspan('1.1.1')   

('1', '<td colspan="2">&nbsp</td>')

In [106]:
query = """ \
	SELECT DISTINCT  
	Q_TypeCode AS FTPE,  IDTEXT,  SortPos, 
	PK_Question [{0}], 
	REPLACE(IDTEXT, '.','_') AS [{1}], 
	CONVERT(NVARCHAR(5), PK_PickListType) AS [{2}],
    '3' AS [{3}],
	CONVERT(NVARCHAR(5), PK_Question) + ' ' + IDTEXT + ' ' +  QTEXT AS [{4}],
	' r_idt_' + REPLACE(IDTEXT, '.','_') +  ' ct_' +  Q_TypeCode  AS [{5}]
	FROM  vwQuestions WHERE PK_FORM LIKE '2022-A-HVA'   ORDER BY SortPos ASC
"""  
df  = get_df(query) 
df[0:4]

,FTPE,IDTEXT,SortPos,{0},{1},{2},{3},{4},{5}
0,LABEL,0.1,1,28200,0_1,None,3,28200 0.1 HVA Information,r_idt_0_1 ct_LABEL
1,TXT,1,2,28201,1,None,3,28201 1 What is the purpose of the HVA?,r_idt_1 ct_TXT
2,MULTICHECKBOX,1.1,3,28202,1_1,356,3,28202 1.1 Please Select the Applicable Purpose...,r_idt_1_1 ct_MULTICHECKBOX
3,TXT,1.1.1,4,28203,1_1_1,None,3,28203 1.1.1 If .GOV Domain Registration System...,r_idt_1_1_1 ct_TXT


In [109]:
st=''
for i,r in df.iterrows():
    tmp = f"C:\_som\_src\{r['FTPE']}.aspx"
    r0 = r['{0}']
    with open(tmp, 'r') as f:
        tmp = f.read()
    csp = cspan(r["IDTEXT"])
    tmp = re.sub('colspan="\d"','colspan="{3}"',tmp)
    tmp = tmp.replace("<!--{cols}-->", str(csp[1])) 
    tmp = tmp.replace("{3}", str(csp[0])) 
    tmp = tmp.replace("{0}", str(r["{0}"])) 
    tmp = tmp.replace("{1}", str(r["{1}"])) 
    tmp = tmp.replace("{2}", str(r["{2}"]))  
    tmp = tmp.replace("{4}", str(r["{4}"])) 
    tmp = tmp.replace("{5}", str(r["{5}"]))   
    st=st+tmp
with open(r'C:\Users\Tim\Documents\1802\HVA\page.aspx', 'w') as f:
    f.write(st)
print(st)

<!-- 
    28200 0.1 HVA Information
--->
<tr id="r-m-0_1" class=" r_idt_0_1 ct_LABEL">
    
    <td class="LabelColumn boldtext" colspan="6">
        <uc1:CBlabel ID="CBlabel_28200" PK_key="28200" runat="server" LabelType="Instructions" CSSClass="CustomControlLabelNoVertAlign" />
    </td>
</tr>
<!-- 
    28201 1 What is the purpose of the HVA?
--->
<tr id="r-m-1" class=" r_idt_1 ct_TXT" >
    
    <td class="LabelNoWrap" style="width: 4%; ">
        <uc1:CBlabel ID="CBlabel_1_i"  LinkEditID="CBtext_1" runat="server" LabelType="QuestionNumber" CSSClass="CustomControlLabelNoVertAlign"   />
    </td>
    <td class="LabelColumn" colspan="3" style="width: 73%; ">
        <uc1:CBlabel ID="CBlabel_1_ii" LinkEditID="CBtext_1" runat="server"  CSSClass="CustomControlLabelNoVertAlign" LabelType="Question" /> 
        <asp:Label ID="Label_1_i" runat="server" Text="" CssClass="CustomControlLabelNoVertAlign"></asp:Label>
    </td>
    <td class="ControlColumn" style="width: 17%; ">
        <uc2:CBt

In [4]:
PK_Question="{0}"
ID="{1}"
PickListID="{2}"
COMMENTS="{4}"
CLS="{5}" 

aspTXT = ''
with open(fr'C:\_som\_src\MULTICHECKBOX.aspx', 'r') as f:
    aspTXT = f.read()
print(aspTXT)    

<!-- 
    {4}
--->
<tr id="r-m-{1}" class="{5}" >
    <td class="LabelNoWrap" style="width: 4%; text-align: left">
        <uc1:CBlabel CSSClass="CustomControlLabelNoVertAlign" ID="CBlabel_{1}_i" runat="server" LabelType="QuestionNumber"
            LinkEditID="CbPickListMultiSelect_{1}" />
    </td>
    <td class="LabelColumn" colspan="3" style="width: 73%; text-align: left">
        <uc1:CBlabel CSSClass="CustomControlLabelNoVertAlign" ID="CBlabel_{1}_ii" runat="server" LabelType="Question"
            LinkEditID="CbPickListMultiSelect_{1}" /> 
        <asp:Label ID="Label_{1}_i" runat="server" Text="" CssClass="CustomControlLabelNoVertAlign"></asp:Label>
    </td>
    <td class="ControlColumn" style="width: 17%; text-align: right">
        <uc9:CbPickListMultiSelect ID="CbPickListMultiSelect_{1}" runat="server" CSSClass="CustomControlValue" EditState="EditMode" 
            DoPostBack="true" PK_Question="{0}" PK_PickListType="{2}" SelfLoadingOption="PicklistByPK"  />
    </td>
    <

In [3]:


df = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='PurposePickList').fillna('NULL') 
 


(4250,@PKT, N'GVDMNGNY',N'.GOV Domain Registration Systems',0, GETDATE(), 1),
(4251,@PKT, N'CCDYM',N'Access to Data systems',1, GETDATE(), 1),
(4252,@PKT, N'GNCYPPL',N'Agency support tool ',2, GETDATE(), 1),
(4253,@PKT, N'BNNLLGNC',N'Business Intelligence Services',3, GETDATE(), 1),
(4254,@PKT, N'CLLBNLHP',N'Collaboration Tools (SharePoint)',4, GETDATE(), 1),
(4255,@PKT, N'CLLCNLYZ',N'Collect, analyze, maintain and share data',5, GETDATE(), 1),
(4256,@PKT, N'CNLDBV',N'Consulate Database (VISA)',6, GETDATE(), 1),
(4257,@PKT, N'CYBCYMNN',N'Cybersecurity Monitoring',7, GETDATE(), 1),
(4258,@PKT, N'CYBCYLLN',N'Cybersecurity tools/solutions',8, GETDATE(), 1),
(4259,@PKT, N'DPY',N'Data Repository',9, GETDATE(), 1),
(4260,@PKT, N'DCNMKNGL',N'Decision making tool',10, GETDATE(), 1),
(4261,@PKT, N'DPLMCNDN',N'Diplomatic and Incident Response Communications',11, GETDATE(), 1),
(4262,@PKT, N'MLVCNDNN',N'Email services and/or internet access',12, GETDATE(), 1),
(4263,@PKT, N'MGNCYMNG',N'Emergency 